<a href="https://colab.research.google.com/github/Vampaxx/Pyspark_basics/blob/main/groupBy()_groupBy_agg().ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0eb483d63daa58f45c2fa471d65c17511904d750a926709eaeda6a3424b77a07
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [48]:
from pyspark.sql.types import (StringType,StructField,
                               StructType,IntegerType,
                               ArrayType,MapType)
from pyspark.sql import SparkSession

from pyspark.sql.functions import col,explode,split
from pyspark.sql.functions import col,lit,when,expr,avg,max,round



In [3]:
spark  = SparkSession.builder.appName('groupBy &groupByagg').getOrCreate()

## GroupBy()

Its similar to SQL GROUP BY clause, pyspark groupBy() function is used to collect the identical data into groups on Dataframe and perform count, sum,avg,min, max on the grouped data

In [4]:
data = [
    (1,'Micheal','Male',2312,'IT'),\
    (2,'Jackson','Male',1245,'HR'),\
    (3,'Mithila','Female',1451,'HR'),\
    (4,'Daniel','Female',641,'R&D'),\
    (5,'Johnson','Male',5645,'Payroll'),\
    (6,'Rajeev','Male',1245,'HR'),\
    (7,'Aishwarya','Female',5645,'IT'),\
    (8,'Rachel','Female',2345,'R&D'),\
    (9,'Akhil sharma', 'Male',35789,'Payroll')
]

schema = ['Emp_id','Emp_name','Gender','Salary','Department']

df = spark.createDataFrame(data,schema)
df.show()


+------+------------+------+------+----------+
|Emp_id|    Emp_name|Gender|Salary|Department|
+------+------------+------+------+----------+
|     1|     Micheal|  Male|  2312|        IT|
|     2|     Jackson|  Male|  1245|        HR|
|     3|     Mithila|Female|  1451|        HR|
|     4|      Daniel|Female|   641|       R&D|
|     5|     Johnson|  Male|  5645|   Payroll|
|     6|      Rajeev|  Male|  1245|        HR|
|     7|   Aishwarya|Female|  5645|        IT|
|     8|      Rachel|Female|  2345|       R&D|
|     9|Akhil sharma|  Male| 35789|   Payroll|
+------+------------+------+------+----------+



In [10]:
df = spark.read.csv(path='/content/drive/MyDrive/Pyspark/datas/Clean_Dataset.csv',header=True,inferSchema=True)
df = df.withColumnRenamed('_c0','Id')
df.show(5)

+---+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
| Id| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+---+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|  0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|  1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|  2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|  3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|  4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+---+--------+-------+-----------+------

In [39]:
## max price of ticket based on the airline company

df.groupBy('airline')\
  .max('price').show()

+---------+----------+
|  airline|max(price)|
+---------+----------+
|   Indigo|     31952|
| SpiceJet|     34158|
|Air_India|     90970|
|  AirAsia|     31917|
| GO_FIRST|     32803|
|  Vistara|    123071|
+---------+----------+



## Renaming the aggregate column


In [46]:
## method 1

df.groupBy('airline')\
  .max('price')\
  .withColumnRenamed('max(price)','max_ticket_price').show()

## method 2

result_df = df.groupBy("airline")\
    .agg(max('price').alias('max_ticket_price'))

result_df.show()

+---------+----------------+
|  airline|max_ticket_price|
+---------+----------------+
|   Indigo|           31952|
| SpiceJet|           34158|
|Air_India|           90970|
|  AirAsia|           31917|
| GO_FIRST|           32803|
|  Vistara|          123071|
+---------+----------------+

+---------+----------------+
|  airline|max_ticket_price|
+---------+----------------+
|   Indigo|           31952|
| SpiceJet|           34158|
|Air_India|           90970|
|  AirAsia|           31917|
| GO_FIRST|           32803|
|  Vistara|          123071|
+---------+----------------+



## Returning multiple aggregate in same dataframes

In [51]:


df.groupBy('airline').agg(
    round(max('price'),2).alias('max_ticket_price'),
    round(avg('price'),2).alias('avg_ticket_price')
).show()

+---------+----------------+----------------+
|  airline|max_ticket_price|avg_ticket_price|
+---------+----------------+----------------+
|   Indigo|           31952|         5324.22|
| SpiceJet|           34158|         6179.28|
|Air_India|           90970|        23507.02|
|  AirAsia|           31917|         4091.07|
| GO_FIRST|           32803|         5652.01|
|  Vistara|          123071|        30396.54|
+---------+----------------+----------------+



## Filtering on aggregate (like SQL HAVING statement)

In [55]:
df.groupBy('airline').agg(
    round(max('price'),2).alias('max_ticket_price'),
    round(avg('price'),2).alias('avg_ticket_price')
).where(col('avg_ticket_price')>=10000).show()

+---------+----------------+----------------+
|  airline|max_ticket_price|avg_ticket_price|
+---------+----------------+----------------+
|Air_India|           90970|        23507.02|
|  Vistara|          123071|        30396.54|
+---------+----------------+----------------+



## Grouping by multiple columns

In [61]:
df.groupBy(['airline','class']).agg(
    round(avg('duration'),2).alias('avg_duration_of_flight')
).show()


+---------+--------+----------------------+
|  airline|   class|avg_duration_of_flight|
+---------+--------+----------------------+
|  AirAsia| Economy|                  8.94|
|Air_India| Economy|                  16.1|
| GO_FIRST| Economy|                  8.76|
|  Vistara| Economy|                 13.44|
| SpiceJet| Economy|                 12.58|
|   Indigo| Economy|                   5.8|
|Air_India|Business|                 14.63|
|  Vistara|Business|                  13.2|
+---------+--------+----------------------+

